Sources:
- https://developers.google.com/earth-engine/datasets/catalog/GRIDMET_DROUGHT
- Michael and Juan's tutorial on extracting values

In [55]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import altair as alt
import numpy as np

ee.Initialize()

In [8]:
def pd_shp_to_ee_poly(shp):
    """Converts Polygon from GeoPandas to a ee.Geometry.Polygon
    object suitable for use within Google Earth Engine."""
    xs, ys = shp.exterior.coords.xy
    shp_list = [[x, y] for x, y in zip(xs, ys)]
    roi = ee.Geometry.Polygon(shp_list, None, False)
    return roi


def maskClasses(image, vals_to_keep):
    """Masks values of the image to only include those
    within vals_to_keep."""
    masks = []
    finalMask = ee.Image(0)

    for val in vals_to_keep:
        masks.append(image.eq(val))
    
    for mask in masks:
        finalMask = finalMask.Or(mask) 
    
    return image.updateMask(finalMask)


def get_county_roi(county_name):
    """Returns a ee.Geometry.Polygon object representing
    a particular county within Georgia, along with the
    centroid of that object."""
    ga_counties = gpd.read_file("ga-counties/Counties_Georgia.shp")
    county_shp = ga_counties[ga_counties["NAME10"] == county_name].geometry.values[0]
    xs, ys = county_shp.centroid.coords.xy
    county_roi = pd_shp_to_ee_poly(county_shp).simplify(maxError = 1)

    return county_roi, (xs[0], ys[0])


def union_polygons(poly_ls):
    basePoly = poly_ls[0]
    for poly in poly_ls[1:]:
        basePoly = basePoly.union(poly, maxError = 1)

    return basePoly


def rect_from_corners(tl, br):
    return ee.Geometry.Polygon(
        [[tl[0], tl[1]],
        [br[0], tl[1]],
        [br[0], br[1]],
        [tl[0],br[1]]], None, False
    )

In [9]:
# Define Lake Lanier region
tl = (-84.1806793, 34.4159734)
br = (-83.6986542, 34.0919046)
lake_lanier_roi = rect_from_corners(tl, br)

In [10]:
def landsat_roi(roi, bands, fname):
    collection = geemap.landsat_timeseries(
        roi = roi, 
        start_year = 1985, end_year = 2021, 
        start_date = '06-10', end_date = '09-20'
    )

    video_args = {
        'dimensions': 768,
        'region': roi,
        'framesPerSecond': 2,
        'bands': bands,
        'min': 0,
        'max': 4000,
        'gamma': [1, 1, 1]
    }

    geemap.download_ee_video(collection, video_args, fname)
    geemap.add_text_to_gif(
        fname, fname, xy=('92%', '3%'), 
        text_sequence = 1985, font_size = 20, 
        font_color = '#ffffff', 
        add_progress_bar = False
    )

bands = ['Red', 'Green', 'Blue']
landsat_roi(lake_lanier_roi, bands, 'vids/lake-laneer/visible.gif')

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/lake-laneer/visible.gif


![](vids/lake-laneer/visible.gif)

In [11]:
bands = ['SWIR2', 'SWIR1', 'Blue']
landsat_roi(lake_lanier_roi, bands, 'vids/lake-laneer/geology.gif')

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/lake-laneer/geology.gif


![](vids/lake-laneer/geology.gif)

In [13]:
def image_hist(geom):
    def _image_hist(img):
        stat = img.reduceRegion(
            ee.Reducer.frequencyHistogram(),
            bestEffort = True
        )
        return ee.Feature(geom, stat).set({'millis': img.date().millis()})

    return _image_hist

def fc_to_dict(fc):
    prop_names = fc.first().propertyNames()
    prop_lists = fc.reduceColumns(
        reducer=ee.Reducer.toList().repeat(prop_names.size()),
        selectors=prop_names).get('list')

    return ee.Dictionary.fromLists(prop_names, prop_lists)

In [14]:
# Looking at values produced from the particular image collection looking at
# water layers of the bands 
water_collection = (
    ee.ImageCollection('JRC/GSW1_3/MonthlyHistory')
    .select('water')
    .filterBounds(lake_lanier_roi)
    .map(lambda image: image.clip(lake_lanier_roi))
)

get_water_values = image_hist(lake_lanier_roi)

water_stat_fc = ee.FeatureCollection(
    water_collection.map(get_water_values)
)

water_stat = fc_to_dict(water_stat_fc).getInfo()

In [74]:
water_df = pd.DataFrame(water_stat)
water_df_wvals = water_df.water.apply(pd.Series)
water_dfl = pd.concat([water_df, water_df_wvals], axis = 1)
water_dfl = water_dfl[['system:index', '0', '1', '2']]
water_dfl['system:index'] = pd.to_datetime(water_dfl['system:index'], format = "%Y_%m")
water_dfl = water_dfl.rename(columns = {
    "system:index": "date", "0": "no_data", "1": "no_water", "2": "water"
})
water_dfl = water_dfl.fillna(0)
dq_flag = 0.8
water_dfl['total_pixels'] = water_dfl['no_data'] + water_dfl['no_water'] + water_dfl['water']
water_dfl['total_obs'] = water_dfl['no_water'] + water_dfl['water']
water_dfl['prop_water'] = water_dfl['water'] / water_dfl['total_obs']
water_dfl['prop_obs'] = water_dfl['total_obs'] / water_dfl['total_pixels']
water_dfl['prop_water_dq'] = water_dfl['prop_water']
water_dfl.loc[water_dfl['prop_obs'] < dq_flag, 'prop_water_dq'] = np.nan
water_dfl['prop_obs_gt_dq'] = (water_dfl['prop_obs'] >= dq_flag)
water_dfl['prop_obs_gt_dq'] = water_dfl['prop_obs_gt_90'].astype(int).astype(float) * 0.13
water_dfl

,date,no_data,no_water,water,total_pixels,total_obs,prop_water,prop_obs,prop_water_dq,prop_obs_gt_90
0,1984-03-01,2.150816e+06,0.000000e+00,0.000000,2.150816e+06,0.000000e+00,NaN,0.000000,NaN,0.00
1,1984-04-01,2.150816e+06,0.000000e+00,0.000000,2.150816e+06,0.000000e+00,NaN,0.000000,NaN,0.00
2,1984-05-01,2.974798e+04,1.954737e+06,166331.254902,2.150816e+06,2.121068e+06,0.078419,0.986169,0.078419,0.13
3,1984-06-01,1.199624e+03,1.967170e+06,182446.266667,2.150816e+06,2.149617e+06,0.084874,0.999442,0.084874,0.13
4,1984-07-01,4.921812e+05,1.573684e+06,84951.266667,2.150816e+06,1.658635e+06,0.051218,0.771165,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...
437,2020-08-01,1.786959e+04,1.954248e+06,178698.423529,2.150816e+06,2.132947e+06,0.083780,0.991692,0.083780,0.13
438,2020-09-01,1.211608e+05,1.851018e+06,178637.517647,2.150816e+06,2.029655e+06,0.088014,0.943668,0.088014,0.13
439,2020-10-01,4.460000e+02,1.953031e+06,197338.862745,2.150816e+06,2.150370e+06,0.091770,0.999793,0.091770,0.13
440,2020-11-01,2.150816e+06,0.000000e+00,0.000000,2.150816e+06,0.000000e+00,NaN,0.000000,NaN,0.00


In [75]:
# Difficulty: high volatility regions tend to correspond to time periods with
# worse coverage (in terms of pixels for where there is no data)
line = alt.Chart(
    water_dfl[water_dfl['date'] >= '2000-01-01']
).mark_line(size=2).encode(
    x='date:T',
    y='prop_water:Q',
).properties(width=900, height=400)

bg = alt.Chart(
    water_dfl[water_dfl['date'] >= '2000-01-01']
).mark_bar(
    opacity = 0.2,
    color = 'red'
).encode(
    x='date:T',
    y='prop_obs_gt_dq:Q',
)

bg + line

alt.LayerChart(...)

In [76]:
line = alt.Chart(
    water_dfl[water_dfl['date'] >= '2000-01-01']
).mark_line(size=2).encode(
    x='date:T',
    y='prop_water_dq:Q',
).properties(width=900, height=400)

bg = alt.Chart(
    water_dfl[water_dfl['date'] >= '2000-01-01']
).mark_bar(
    opacity = 0.2,
    color = 'red'
).encode(
    x='date:T',
    y='prop_obs_gt_dq:Q',
)

bg + line

alt.LayerChart(...)

In [65]:
alt.Chart(
    water_dfl[water_dfl['date'] >= '2000-01-01']
).mark_bar().encode(
    alt.X("prop_obs:Q", bin=True),
    y='count()',
)

alt.Chart(...)

In [73]:
collection = geemap.create_timeseries(
    collection = ee.ImageCollection('JRC/GSW1_3/MonthlyHistory'),
    start_date = "1990-03-16",
    end_date = "2021-01-01",
    region = lake_lanier_roi,
    bands = "water",
    frequency = "year"
)

video_args = {
    'dimensions': 768,
    'region': lake_lanier_roi,
    'framesPerSecond': 3,
    'bands': ['water'],
    'min': 0.0,
    'max': 2.0,
    'palette': ['000000', 'fffcb8', '0905ff'],
    'bestEffort': True
}

fname = "vids/lake-laneer/water-only.gif"
geemap.download_ee_video(collection, video_args, fname)
geemap.add_text_to_gif(
    fname, fname, xy=('90%', '3%'), 
    text_sequence = 1990, font_size = 20, 
    font_color = '000000', 
    add_progress_bar = False
)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/lake-laneer/water-only.gif


![](vids/lake-laneer/water-only.gif)

In [ ]:
# want to try and do something using the images as a time series themselves, and try and do
# some type of spatial and temporal averaging/pooling to understand what is occuring locally
# over time, and to make more accurate inferences as to what is going on